# Neural Network Training
This notebook implements a neural network model to predict target variables in the soccer dataset. The training pipeline includes data preparation, model training, and evaluation.

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from keras.models import Sequential
from keras.layers import Dense
from keras.callbacks import EarlyStopping
from sklearn.metrics import mean_squared_error, r2_score
import yaml

# Load configuration settings
with open('../../config/training_configs.yaml') as file:
    training_config = yaml.safe_load(file)

with open('../../config/model_configs.yaml') as file:
    model_config = yaml.safe_load(file)

# Set parameters
epochs = training_config['epochs']
batch_size = training_config['batch_size']
learning_rate = model_config['neural_network']['learning_rate']
input_dim = len(training_config['features'])  # Number of features
output_dim = len(training_config['targets'])  # Number of targets

: 

In [ ]:
# Load and preprocess data
df = pd.read_csv('../../data/preprocessed_data.csv')
X = df[training_config['features']].values
y = df[training_config['targets']].values

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Scale features
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

In [ ]:
# Build the neural network model
model = Sequential()
model.add(Dense(64, input_dim=input_dim, activation='relu'))
model.add(Dense(32, activation='relu'))
model.add(Dense(output_dim, activation='linear'))

# Compile the model
model.compile(optimizer='adam', loss='mean_squared_error', metrics=['mae'])

In [ ]:
# Train the model
early_stopping = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)
history = model.fit(X_train, y_train, epochs=epochs, batch_size=batch_size, 
                    validation_split=0.2, callbacks=[early_stopping], verbose=1)

In [ ]:
# Evaluate the model
y_pred_train = model.predict(X_train)
y_pred_test = model.predict(X_test)

train_mse = mean_squared_error(y_train, y_pred_train)
test_mse = mean_squared_error(y_test, y_pred_test)
train_r2 = r2_score(y_train, y_pred_train)
test_r2 = r2_score(y_test, y_pred_test)

print(f'Train MSE: {train_mse:.4f}, Train R²: {train_r2:.4f}')
print(f'Test MSE: {test_mse:.4f}, Test R²: {test_r2:.4f}')

In [ ]:
# Visualize loss curves
plt.figure(figsize=(12, 6))
plt.plot(history.history['loss'], label='Train Loss')
plt.plot(history.history['val_loss'], label='Validation Loss')
plt.title('Loss Curves')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend()
plt.grid(True)
plt.show()